In [1]:
import math
import numpy as np
import tensorflow as tf

from tensorflow.python.ops.rnn_cell import GRUCell
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.python.ops.rnn_cell import DropoutWrapper, ResidualWrapper

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.layers.core import Dense
from tensorflow.python.util import nest

from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.contrib.seq2seq.python.ops import beam_search_decoder
from datetime import datetime

from preprocess import *
from loading_util import *

from sklearn.model_selection import train_test_split

c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Resetter
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
#embedding parameters
embedding_size = 50
vocab_size = 400003

#data parameters
eMax_allowed_length = 3000
#dMax_allowed_length = 15

#network parameters
hidden_units = 64
n_outputs = 5
depth = 2
n_epochs = 5
learning_rate = 0.001
batch_size = 1024

#Saving Parameters
#Tensorboard Logs
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tmp/LSTM_evaluate/tf_logs"
log_dir = "{}/run-{}/".format(root_logdir,now)
#Tensorboard logs end

save_path = 'tmp/LSTM_Evaluate/model/mymodel.ckpt'

In [4]:
#Fetching data
#default directory: 'data/essays.csv'
X,Y= read_csv_essay('data/essays.csv')
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,shuffle=True,train_size = 0.7,test_size=0.3 ,stratify=Y)

In [5]:
#Fetching glove vectors
#default directory: "./glove.6B.50d.txt"
embedding_size = 50
wi,iw,wv = read_glove_vecs()
len(wi)
len(iw)
reset_graph()

In [6]:
#Adding extra tokens to glove dictionary
#IMPORTANT: Don't run more than once
go_index,eos_index,unk_index = add_extra_to_dict(wi,iw,wv,embedding_size)
emb = map_dict_to_list(iw,wv)

In [7]:
#preprocessing data
#Mapping each word in a sentence to its glove index
eInput_train,eLengths_train = fit_essay_text(data= X_train,word_to_index = wi,max_allowed_seq_length = eMax_allowed_length)
eInput_test,eLengths_test = fit_essay_text(data= X_test,word_to_index = wi,max_allowed_seq_length = eMax_allowed_length)

eInput_train = np.array(eInput_train) 
eLengths_train = np.array(eLengths_train)

eInput_test = np.array(eInput_test) 
eLengths_test = np.array(eLengths_test)

print(eInput_train)
print()
print(eInput_test)

[[185456 126551 209644 ... 400001 400001 400001]
 [193715 192972 173235 ... 400001 400001 400001]
 [185456 200034 389216 ... 400001 400001 400001]
 ...
 [308560 264936  60664 ... 400001 400001 400001]
 [185456  71949  88125 ... 400001 400001 400001]
 [385217 177230 185456 ... 400001 400001 400001]]

[[185456 302351 360399 ... 400001 400001 400001]
 [217786 261148 185456 ... 400001 400001 400001]
 [336113 185456 200034 ... 400001 400001 400001]
 ...
 [185456  52942 368716 ... 400001 400001 400001]
 [344461 268045 107571 ... 400001 400001 400001]
 [308560 264936 185456 ... 400001 400001 400001]]


In [8]:
#encoder inputs: [batch_size, max_time_steps]
encoder_inputs = tf.placeholder(dtype = tf.int32, shape = (None,None), name = 'encoder_inputs')
#encoder_inputs_length: [batch_size]
encoder_inputs_length = tf.placeholder(dtype=tf.int32, shape=(None,) , name = 'encoder_inputs_length')

#encoder_outputs
traits = tf.placeholder(tf.float32,[None,n_outputs])

In [9]:

#Instantiating embeddings
embedding_variable = tf.Variable(tf.constant(0.0, shape = [vocab_size, embedding_size]),trainable = True, name = 'embedding')
embedding_placeholder = tf.placeholder(tf.float32, shape=[vocab_size,embedding_size], name = 'embedding_placeholder' )
encoder_embeddings = embedding_variable.assign(embedding_placeholder)

encoder_inputs_embedded=tf.nn.embedding_lookup(encoder_embeddings,encoder_inputs)

In [10]:
#Testing embedding lookup
with tf.Session() as sess:
    embed=sess.run(encoder_inputs_embedded, feed_dict={embedding_placeholder:emb ,encoder_inputs:eInput_train })
    print(embed.shape)
    print(embed[0][0])

(1726, 3000, 50)
[ 1.1891e-01  1.5255e-01 -8.2073e-02 -7.4144e-01  7.5917e-01 -4.8328e-01
 -3.1009e-01  5.1476e-01 -9.8708e-01  6.1757e-04 -1.5043e-01  8.3770e-01
 -1.0797e+00 -5.1460e-01  1.3188e+00  6.2007e-01  1.3779e-01  4.7108e-01
 -7.2874e-02 -7.2675e-01 -7.4116e-01  7.5263e-01  8.8180e-01  2.9561e-01
  1.3548e+00 -2.5701e+00 -1.3523e+00  4.5880e-01  1.0068e+00 -1.1856e+00
  3.4737e+00  7.7898e-01 -7.2929e-01  2.5102e-01 -2.6156e-01 -3.4684e-01
  5.5841e-01  7.5098e-01  4.9830e-01 -2.6823e-01 -2.7443e-03 -1.8298e-02
 -2.8096e-01  5.5318e-01  3.7706e-02  1.8555e-01 -1.5025e-01 -5.7512e-01
 -2.6671e-01  9.2121e-01]


In [11]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = hidden_units)
outputs,states = tf.nn.dynamic_rnn(basic_cell,encoder_inputs_embedded,dtype=tf.float32, sequence_length=encoder_inputs_length)

In [12]:
logits = tf.layers.dense(states,n_outputs)
xentropy = tf.nn.sigmoid_cross_entropy_with_logits(
    labels=traits,
    logits=logits,
    name='CostFunction_Sigmoid'
)
loss = tf.reduce_mean(xentropy)

In [13]:
#Testing Purposes
probs = tf.nn.sigmoid(logits)

In [14]:
#Testing sigmoid

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    index = 20
    p = sess.run(probs,feed_dict = {encoder_inputs: eInput_train[:index],encoder_inputs_length: eLengths_train[:index],traits: Y_train[:index],embedding_placeholder:emb})
    print(p)

[[0.59138316 0.29785445 0.69701904 0.5201288  0.40545353]
 [0.62253946 0.4392154  0.5364552  0.47397876 0.3984339 ]
 [0.5556527  0.37168843 0.51423204 0.54537505 0.49251857]
 [0.54505855 0.42820388 0.48887613 0.4496058  0.3949534 ]
 [0.5945363  0.29630968 0.61203045 0.49414858 0.3346868 ]
 [0.6876622  0.35715917 0.629866   0.5922045  0.4594781 ]
 [0.4966127  0.4259384  0.5504083  0.52324927 0.28461564]
 [0.5684401  0.2702546  0.6603765  0.65046465 0.44695297]
 [0.54228437 0.4494883  0.57085264 0.51626337 0.36013335]
 [0.62732345 0.34435126 0.6176377  0.6136212  0.47191882]
 [0.5180603  0.31977937 0.6466851  0.5676208  0.39892706]
 [0.6547974  0.24939713 0.6854767  0.5681574  0.4597073 ]
 [0.6930299  0.44501805 0.4024151  0.7843428  0.3925475 ]
 [0.62483376 0.30008104 0.6109589  0.5940931  0.44638547]
 [0.6245365  0.34007066 0.6460493  0.6353759  0.38778362]
 [0.5742017  0.4860044  0.67151016 0.42893183 0.36144435]
 [0.46774668 0.40163326 0.62514603 0.37463486 0.44902757]
 [0.6070405  0

In [15]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss) ;
correct = tf.equal(tf.round(probs),tf.round(traits))
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [16]:
#Testing accuracy

init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    index = 20
    c,p = sess.run([correct,accuracy],feed_dict = {encoder_inputs: eInput_train[:index],
                                                   encoder_inputs_length: eLengths_train[:index],
                                                   traits: Y_train[:index],embedding_placeholder:emb})
    print(p)
    print(c)

0.54
[[ True False  True False  True]
 [ True  True  True  True  True]
 [ True False False False  True]
 [ True False  True False  True]
 [ True  True  True False False]
 [False  True False  True  True]
 [False  True  True False False]
 [ True  True  True  True False]
 [False  True  True False False]
 [ True  True  True  True  True]
 [False  True False False False]
 [False False False  True  True]
 [False  True False  True  True]
 [ True False False  True  True]
 [False False  True False False]
 [False False False False  True]
 [ True  True  True False  True]
 [False  True  True  True False]
 [ True  True False False  True]
 [False False False False  True]]


In [17]:
acc_summary = tf.summary.scalar('Accuracy',accuracy)
file_writer = tf.summary.FileWriter(log_dir+'_train',tf.get_default_graph()) 
file_writer_test = tf.summary.FileWriter(log_dir+'_test',tf.get_default_graph()) 

In [18]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [19]:
def fetch_batch(data_x,data_y,lengths,batch_index,batch_size):
    return (data_x[batch_index:batch_index+batch_size,:],
            data_y[batch_index:batch_index+batch_size,:],
            lengths[batch_index:batch_index+batch_size])

In [20]:
total_batches = int(eInput_train.shape[0]/batch_size)


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch_index in range(total_batches):
            batch_X,batch_Y,batch_lengths = fetch_batch(eInput_train,Y_train,eLengths_train,batch_index,batch_size)
            
            sess.run(training_op,feed_dict = {encoder_inputs: batch_X,
                                       encoder_inputs_length: batch_lengths,
                                       traits: batch_Y,
                                       embedding_placeholder:emb})
            acc_train = accuracy.eval(feed_dict = {encoder_inputs: batch_X,
                                       encoder_inputs_length: batch_lengths,
                                       traits: batch_Y,
                                       embedding_placeholder:emb})
            acc_test = accuracy.eval(feed_dict = {encoder_inputs: eInput_test,
                                       encoder_inputs_length: eLengths_test,
                                       traits: Y_test,
                                       embedding_placeholder:emb})
            
            batch_loss = accuracy.eval(feed_dict = {encoder_inputs: batch_X,
                                       encoder_inputs_length: batch_lengths,
                                       traits: batch_Y,
                                       embedding_placeholder:emb})
            
            print('Epoch:',epoch,' Batch_number: ',batch_index,
                  'Batch_loss:',batch_loss, 
                  'TRAIN_accuracy:',acc_train,' TEST_accuracy:',acc_test)
        saver.save(sess,save_path)
        

Epoch: 0  Batch_number:  0 Batch_loss: 0.50996095 TRAIN_accuracy: 0.50996095  TEST_accuracy: 0.5060729
Epoch: 1  Batch_number:  0 Batch_loss: 0.515625 TRAIN_accuracy: 0.515625  TEST_accuracy: 0.505803
Epoch: 2  Batch_number:  0 Batch_loss: 0.51308596 TRAIN_accuracy: 0.51308596  TEST_accuracy: 0.50121456
Epoch: 3  Batch_number:  0 Batch_loss: 0.5115234 TRAIN_accuracy: 0.5115234  TEST_accuracy: 0.4979757
Epoch: 4  Batch_number:  0 Batch_loss: 0.51464844 TRAIN_accuracy: 0.51464844  TEST_accuracy: 0.50013494


In [29]:
#Checking for an input

myAns = 'Its a fine day'
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    candidate_ans = [myAns]
    candidate_input,input_length = fit_encoder_text(data= candidate_ans,
                                                    word_to_index = wi,
                                                    max_allowed_seq_length = eMax_allowed_length)
    p = sess.run(probs,feed_dict = {encoder_inputs: candidate_input,
                                    encoder_inputs_length: input_length,
                                    embedding_placeholder:emb})
    print(p)


[[0.63497984 0.35376385 0.6257002  0.5465952  0.46937215]]


In [30]:
def network_pass(candidate_ans,sess):
    
    candidate_ans = [candidate_ans]
    candidate_input,input_length = fit_encoder_text(data= candidate_ans,
                                                    word_to_index = wi,
                                                    max_allowed_seq_length = eMax_allowed_length)
    p = sess.run(probs,feed_dict = {encoder_inputs: candidate_input,
                                    encoder_inputs_length: input_length,
                                    embedding_placeholder:emb})
    return p
    

In [33]:
init = tf.global_variables_initializer()
def activate_bot():
    with tf.Session() as sess:
        sess.run(init)
        saver.restore(sess,save_path)
        
        count = 1 
        Q = 'Tell me about yourself.'
        while(count!=5):
            candidate_ans = input(Q)
            Q = network_pass(candidate_ans,sess)
            print()
            print(Q)
            count+=1
        
activate_bot()

INFO:tensorflow:Restoring parameters from tmp/LSTM_Evaluate/model/mymodel.ckpt
Tell me about yourself.Hi i am gitesh khanna and i love working in machine learning.

[[0.414144   0.5792102  0.56126016 0.40126243 0.5694154 ]]
[[0.414144   0.5792102  0.56126016 0.40126243 0.5694154 ]]What makes you stand out from the other candidates?i am a hardworkinh person who is passionate to gain more knowledge. I am a quick learner and I can easily handle multiple tasks in minimum supervision.

[[0.5380904  0.41997582 0.5488085  0.47600383 0.5570909 ]]
[[0.5380904  0.41997582 0.5488085  0.47600383 0.5570909 ]]What makes you angry?Anger is a term naturally associated with everybody I guess. I am not known to get angry normally but there are instances when I did become angry, particularly in case of working in teams if I find that fellow members are taking full credit for a work in which they didn't contribute.

[[0.5119227  0.51350206 0.5206798  0.40508813 0.53584296]]
[[0.5119227  0.51350206 0.52067